<a href="https://colab.research.google.com/github/kavyajeetbora/end_to_end_gee_with_python/blob/master/end_to_end_earth_engine/Module_03_Supervised_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 3: Supervised Classification

## Introduction to Machine Learning and Supervised Classification

Supervised classification is arguably the most important classical machine learning techniques in remote sensing. Applications range from generating Land Use/Land Cover maps to change detection. Google Earth Engine is unique suited to do supervised classification at scale. The interactive nature of Earth Engine development allows for iterative development of supervised classification workflows by combining many different datasets into the model. This module covers basic supervised classification workflow, accuracy assessment, hyperparameter tuning and change detection


01. Basic Supervised Classification
02. Accuracy Assessment
03. Improving the Classification
04. Exporting Classification Results
05. Calculating Area

In [1]:
import ee
import geemap

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

## Basic Supervised Classification

- We will learn how to do a basic land cover classification using training - samples collected from the Code Editor using the High Resolution basemap imagery provided by Google Maps.
- This method requires no prior training data and is quite effective to generate high quality classification samples anywhere in the world.
- The goal is to classify each source pixel into one of the following classes - urban, bare, water or vegetation.
- Using the drawing tools in the code editor, you create 4 new feature collection with points representing pixels of that class.
- Each feature collection has a property called landcover with values of 0, 1, 2 or 3 indicating whether the feature collection represents urban, bare, water or vegetation respectively.
- We then train a Random Forest classifier using these training set to build a model and apply it to all the pixels of the image to create a 4 class image

**Fun fact**: The classifiers in Earth Engine API have names starting with smile - such as ee.Classifier.smileRandomForest(). The smile part refers to the [Statistical Machine Intelligence and Learning Engine (SMILE)](https://haifengl.github.io/index.html) JAVA library which is used by Google Earth Engine to implement these algorithms.

<img src="https://courses.spatialthoughts.com/images/end_to_end_gee/classified.png" height=300/>

### Load the geomtry of area of interest:

In [18]:
bangalore = ee.FeatureCollection('users/ujavalgandhi/public/bangalore_boundary')
geometry = bangalore.geometry()
bangalore.size()

### Load the sentinel image

The band values will be used as the input properties for the classifier

In [19]:
## Get the sentinel image collection
s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

## Filter the image collection
filtered = s2.filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",30))\
.filter(ee.Filter.date('2023-01-01','2024-01-01'))\
.filter(ee.Filter.bounds(geometry))

filtered.size()

Since this is a image collection, convert it to a composite using `median()`

In [20]:
## create a composite and clip it as per geometry
median = filtered.median().clip(geometry)

### Load the GCPs
Now load the gcps (ground control points) that are labelled with the correct class: urban, water, bare and vegetation

In [21]:
## Load the training data with lables
## basically a point geometry with a class value store as property: 'landcover'
urban = ee.FeatureCollection('users/ujavalgandhi/e2e/urban_gcps')
water = ee.FeatureCollection('users/ujavalgandhi/e2e/water_gcps')
bare = ee.FeatureCollection('users/ujavalgandhi/e2e/bare_gcps')
vegetation = ee.FeatureCollection('users/ujavalgandhi/e2e/vegetation_gcps')

## Now merge the data into one single table
gcps = urban.merge(water).merge(bare).merge(vegetation)
## Viewing the first gcp
gcps.getInfo()['features'][0]

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [77.65618319730623, 12.954774480921643]},
 'id': '1_1_1_00000000000000000000',
 'properties': {'landcover': 0}}

### Prepare the training data

In [22]:
## Prepare the training data
## we need to merge the pixel values from sentinel image with the class values
## The band values from sentinel will be the input values
## and gcps landcover property will be the output class
training = median.sampleRegions(
    collection = gcps,
    properties = ['landcover'],
    scale=100
)

Here the all the bands are the input characteristics of the pixel and 'landcover' value is the class that determines whether it is urban, water, bare or vegetation

### Plot the training data on a map

In [23]:
## Plot on map
vizParams = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', "B2"]
}

Map = geemap.Map()
Map.addLayer(median, vizParams, name='Sentinel')
Map.addLayer(gcps, {'color':'red'})
Map.centerObject(geometry, zoom=12)

Map

Map(center=[12.978777310922773, 77.60355759502113], controls=(WidgetControl(options=['position', 'transparent_…

### Train a classifier

We will use a random forest model for training

In [24]:
classifier = ee.Classifier.smileRandomForest(50)
classifier

train the model

In [25]:
classifier = classifier.train(
    features= training,
    classProperty = 'landcover',
    inputProperties = median.bandNames()
)

classifier

In [26]:
classified = median.classify(classifier)

In [27]:
palette = ['#e41a1c','#377eb8','#4daf4a','#984ea3']
visParams = {
    'min': 0,
    'max':3,
    'palette': palette
}

Map = geemap.Map()
Map.addLayer(classified, visParams, name="prediction")
Map.centerObject(geometry, zoom=12)
Map

Map(center=[12.978777310922773, 77.60355759502113], controls=(WidgetControl(options=['position', 'transparent_…

Filter the water area

In [28]:
water = classified.eq(2)
Map = geemap.Map()
Map.addLayer(water, visParams, name="water")
Map.centerObject(geometry, zoom=12)
Map

Map(center=[12.978777310922773, 77.60355759502113], controls=(WidgetControl(options=['position', 'transparent_…

Reference
1. [Module 4 - 01 Basic Supervised Classification - GEE for Water Resources Management](https://youtu.be/Karfbita0Qo?si=M_UpqyqY-mgrGQ3Y)

2. [Module 4 - 03 Accuracy Assessment - GEE for Water Resources Management](https://youtu.be/erwxur0HMao?si=STjsgdeVvJZ7-RzW)

## Accuracy Assessment

It is important to get a quantitative estimate of the accuracy of the classification. To do this, a common strategy is to divide your training samples into 2 random fractions - one used for training the model and the other for validation of the predictions. Once a classifier is trained, it can be used to classify the entire image. We can then compare the classified values with the ones in the validation fraction. We can use the ee.Classifier.confusionMatrix() method to calculate a Confusion Matrix representing expected accuracy.

Classification results are evaluated based on the following metrics

- Overall Accuracy: How many samples were classified correctly.
- Producer’s Accuracy: How well did the classification predict each class.
- Consumer’s Accuracy (Reliability): How reliable is the prediction in each class.
- Kappa Coefficient: How well the classification performed as compared to random assignment.

<img src='https://courses.spatialthoughts.com/images/end_to_end_gee/accuracy_assessment.png' height=300/>